In [313]:
import copy
import operator
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import minmax_scale
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.random import sample_without_replacement

In [314]:
iris = datasets.load_iris()
X, y = iris.data[:, 1:3], iris.target
print X.shape
print y.shape

(150, 2)
(150,)


In [315]:
np.random.seed(123)

def sow(list):
    """sum of class weights"""
    s = 0
    for i in range(len(list)):
        s += np.array(list[i][0])
    return s


def CNE(X,y):
    """CNE create new expert"""
    clf = RandomForestClassifier()
    sample = sample_without_replacement(150,60)
    A = X[sample,:]
    b = y[sample]
    return clf.fit(A,b)


beta = 0.5

#pool of experts
e = [] 

# create first expert
e.append(copy.deepcopy(CNE(X,y)))
w = [1]
wsum = []


for i in range(0,np.size(X,0)):
    for j in range(len(e)):
        llocal = e[j].predict(X[i,:].reshape(1, -1))[0] 
        if llocal!=y[i]:
            w[j] = w[j]*beta
            wsum.append(e[j].predict_proba(X[i,:].reshape(1, -1))*w[j])

    if len(wsum):
        index, value = max(enumerate(sow(wsum)), key=operator.itemgetter(1))
        lglobal = index
    else:
        lglobal = e[j].predict(X[i,:].reshape(1, -1))[0]
        
    if lglobal != y[i]:
        m += 1
        e.append(copy.deepcopy(CNE(X,y)))
        w += [1]
    #scaling weights, w = minmax_scale(w)
    #print w
    #train single examples
    wsum = []
print w

[0.000244140625, 0.03125, 0.000244140625, 0.00048828125, 0.0078125, 0.001953125, 0.0078125, 0.015625, 0.00390625, 0.03125, 0.5, 0.0078125, 0.5, 0.25, 0.125, 0.03125, 0.125, 1, 0.0625, 1, 0.0078125, 0.5, 0.25, 0.5, 1, 1, 0.5, 0.5, 0.25, 0.5, 0.5, 0.5, 1, 1]
